## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，每个TODO计1-2分，共9分，预计代码量30行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 实验报告(python)/用于说明实验的文字块(jupyter notebook)不额外计分，但不写会导致扣分。

注：本任务仅在短句子上进行效果测试，因此对概率的计算可直接进行连乘。在实践中，常先对概率取对数，将连乘变为加法来计算，以避免出现数值溢出的情况。

导入HMM参数，初始化所需的起始概率矩阵，转移概率矩阵，发射概率矩阵

In [1]:
import pickle
import numpy as np

In [2]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的ord作为行key
start_probability = hmm_parameters["start_prob"]  # shape(2,)
trans_matrix = hmm_parameters["trans_mat"]  # shape(2, 2)
emission_matrix = hmm_parameters["emission_mat"]  # shape(2, 65536)

定义待处理的句子

In [3]:
# TODO: 将input_sentence中的xxx替换为你的姓名（1分）
input_sentence = "孙一林是一名优秀的学生"


实现viterbi算法，并以此进行中文分词

In [4]:
def viterbi(sent_orig, start_prob, trans_mat, emission_mat):
    """
    viterbi算法进行中文分词

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        str - 中文分词的结果
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]
    
    # `dp`用来储存不同位置每种标注（B/I）的最大概率值
    dp = np.zeros((2, len(sent_ord)), dtype=float)
    
    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    path = np.zeros((2, len(sent_ord)), dtype=int)
    
    #  TODO: 第一个位置的最大概率值计算（1分） 
    dp[0][0] = start_prob[0]*emission_mat[0][sent_ord[0]]
    dp[1][0] = start_prob[1]*emission_mat[1][sent_ord[0]]
    
    #  TODO: 其余位置的最大概率值计算（填充dp和path矩阵）（2分）
    for i in range(1, len(sent_ord)):
        dp[0][i] = max(dp[0][i-1]*trans_mat[0][0], dp[1][i-1]*trans_mat[1][0]) * emission_mat[0][sent_ord[i]]
        dp[1][i] = max(dp[0][i-1]*trans_mat[0][1], dp[1][i-1]*trans_mat[1][1]) * emission_mat[1][sent_ord[i]]
        if dp[0][i-1]*trans_mat[0][0] >= dp[1][i-1]*trans_mat[1][0]:
            path[0][i] = 0 
        else:
            path[0][i] = 1
        if dp[0][i-1]*trans_mat[0][1] >= dp[1][i-1]*trans_mat[1][1]:
            path[1][i] = 0 
        else:
            path[1][i] = 1
    
    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]
    
    #  TODO：计算labels每个位置上的值（填充labels矩阵）（1分）
    if dp[0][-1] <= dp[1][-1]:
        labels[-1] = 1 
    for i in reversed(range(0, len(sent_ord)-1)):
        labels[i] = path[labels[i + 1]][i + 1]
    
    #  根据lalels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str


viterbi算法简要说明：

第一步：根据已有的起始概率矩阵，转移概率矩阵的参数，由已经转化好的数字表示的汉字首元素，
根据转移矩阵给出的概率值，算出动态规划需要维护数组dp的边界元素值

第二步：对于sent_ord中的每一个元素，取出dp数组中的上一个概率值，根据转移概率矩阵计算概率的累乘（根据提示，在我们这里的短句子条件下，概率是可以直接乘算的），由于我们要保持最优子结构，这里要取最大值。之后再根据对应的数组下标从发射矩阵中取出相应的概率值乘算即可；在更新dp数组之后，也要根据所采用的的概率值，标记相应的路径，以便供下文更新label

第三步：labels矩阵需要反向填充，具体来说，首先初始化好labels数组最后一个元素的值。由于给出的代码中labels初始化的元素均为零，这里就只需要检查dp数组最后一个元素的概率值并进行更新即可，之后仍然是递推地更新整个数组。


In [5]:
print("viterbi算法分词结果：", viterbi(input_sentence, start_probability, trans_matrix, emission_matrix))

viterbi算法分词结果： 孙一/林是/一名/优秀/的/学生/


实现前向算法，计算该句子的概率值

In [6]:
def compute_prob_by_forward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    前向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 初始位置概率的计算（1分）
    dp[0][0] = start_prob[0]*emission_mat[0][sent_ord[0]]
    dp[1][0] = start_prob[1]*emission_mat[1][sent_ord[0]]
    
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    for i in range(1, len(sent_ord)):
        dp[0][i] = (dp[0][i-1]*trans_mat[0][0] + dp[1][i-1]*trans_mat[1][0]) * emission_mat[0][sent_ord[i]]
        dp[1][i] = (dp[0][i-1]*trans_mat[0][1] + dp[1][i-1]*trans_mat[1][1]) * emission_mat[1][sent_ord[i]]
        

    return sum([dp[i][len(sent_ord)-1] for i in range(2)])

实现后向算法，计算该句子的概率值

In [7]:
def compute_prob_by_backward(sent_orig, start_prob, trans_mat, emission_mat):
    """
    后向算法，计算输入中文句子的概率值

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 终末位置概率的初始化（1分）
    dp[0][-1] = 1
    dp[1][-1] = 1
    
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后return概率值（1分）
    for i in reversed(range(0, len(sent_ord)-1)):
        sent_ord_idx = sent_ord[i+1]
        dp[0][i] = trans_mat[0][0]*dp[0][i+1]*emission_mat[0][sent_ord_idx] + trans_mat[0][1]*dp[1][i+1]*emission_mat[1][sent_ord_idx]
        dp[1][i] = trans_mat[1][0]*dp[0][i+1]*emission_mat[0][sent_ord_idx] + trans_mat[1][1]*dp[1][i+1]*emission_mat[1][sent_ord_idx]

    return sum([dp[i][0] * start_prob[i] * emission_mat[i][sent_ord[0]] for i in range(2)])

前后向概率算法的基本思想和viterbi算法都是动态规划的算法，区别在于dp矩阵更新的方法不同。不过，compute_prob_by_forward算法中
dp矩阵的初始化方法是与viterbi完全一致的。

按照助教在群里的建议，我测试了代码的正确性，将姓名改为“小明”之后，viterbi算法分词结果是： “小明/是/一名/优秀/的/学生/“
前向算法概率为1.9261686664194958e-27 后向算法概率为：1.926168666419496e-27。

接下来的代码块展示的是使用我自己姓名的计算结果。


In [8]:
print("前向算法概率：", compute_prob_by_forward(input_sentence, start_probability, trans_matrix, emission_matrix))
print("后向算法概率：", compute_prob_by_backward(input_sentence, start_probability, trans_matrix, emission_matrix))

前向算法概率： 2.17616193949256e-31
后向算法概率： 2.1761619394925606e-31


## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有7处TODO需要填写，每个TODO计1-2分，共9分，预计代码量50行；
2. 允许自行修改、编写代码完成，对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分；
3. 实验报告(python)/用于说明实验的文字块(jupyter notebook)不额外计分，但不写会导致扣分。

构建空格分词器，将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式。例如`apple`将变为`a p p l e </w>`。

In [9]:
import re
import functools

In [10]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")

def white_space_tokenize(corpus):
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到 [["i", "am", "happy"], ["i", "have", "N", "apples"]]

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成
    """

    tokeneds = [list(
        filter(lambda tkn: len(tkn)>0, _splitor_pattern.split(_digit_pattern.sub("N", stc.lower())))) for stc in corpus
    ]
    
    return tokeneds

编写相应函数构建BPE算法需要用到的初始状态词典

In [11]:
def build_bpe_vocab(corpus):
    """
    将语料进行white_space_tokenize处理后，将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的字典，例如：
    输入 corpus=["I am happy.", "I have 10 apples!"]，
    得到
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
     }

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)

    bpe_vocab = dict()
    
    # TODO: 完成函数体（1分）
    for sentence in tokenized_corpus:
        for word in sentence:
            key = ''
            for letter in word:
                key += (letter + ' ')
            key += '</w>'
            bpe_vocab[key] = bpe_vocab.get(key, 0) + 1
    return bpe_vocab

In [12]:
#此函数按照给出的提示直接实现即可，以下是简单的正确性测试：
test_corpus=["I am happy.", "I have 10 apples!"]
test_bpe_vocab = build_bpe_vocab(test_corpus)
for elem in test_bpe_vocab.items():
    print(elem)

('i </w>', 2)
('a m </w>', 1)
('h a p p y </w>', 1)
('h a v e </w>', 1)
('N </w>', 1)
('a p p l e s </w>', 1)


编写所需的其他函数

In [13]:
def get_bigram_freq(bpe_vocab):
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：
    输入 bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple(str, str), int] - "bigram->频数"的词典
    """

    bigram_freq = dict()
    
    # TODO: 完成函数体（1分）
    for word, freq in bpe_vocab.items():
        word_split = word.split(' ')
        for i in range(len(word_split) - 1):
            key = tuple(word_split[i:i + 2])
            bigram_freq[key] = bigram_freq.get(key,0)+freq

    return bigram_freq

In [14]:
#此函数同样直接按照助教所给提示实现即可
#沿用上个正确性测试得到的test_bpe_vocab，继续本问的正确性测试，其结果如下
test_bigram_freq = get_bigram_freq(test_bpe_vocab)
for elem in test_bigram_freq.items():
    print(elem)

(('i', '</w>'), 2)
(('a', 'm'), 1)
(('m', '</w>'), 1)
(('h', 'a'), 2)
(('a', 'p'), 2)
(('p', 'p'), 2)
(('p', 'y'), 1)
(('y', '</w>'), 1)
(('a', 'v'), 1)
(('v', 'e'), 1)
(('e', '</w>'), 1)
(('N', '</w>'), 1)
(('p', 'l'), 1)
(('l', 'e'), 1)
(('e', 's'), 1)
(('s', '</w>'), 1)


In [15]:
def refresh_bpe_vocab_by_merging_bigram(bigram, old_bpe_vocab):
    """
    在"单词分词状态->频数"的词典中，合并指定的bigram（即去掉对应的相邻unigram之间的空格），最后返回新的词典，例如：
    输入 bigram=('i', '</w>')，old_bpe_vocab=
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    得到
    {
        'i</w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }

    Args:
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """
    
    new_bpe_vocab = dict()

    # TODO: 完成函数体（1分）
    for key, freq in old_bpe_vocab.items():
        key = key.replace(" ".join(bigram), "".join(bigram))
        new_bpe_vocab[key] = new_bpe_vocab.get(key, 0) + freq
    
    return new_bpe_vocab

In [16]:
##根据函数说明中提供的例子，此函数的简单正确性测试如下
bigram=('i', '</w>')
old_bpe_vocab={
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
new_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(bigram, old_bpe_vocab)
for elem in new_bpe_vocab.items():
    print(elem)

('i</w>', 2)
('a m </w>', 1)
('h a p p y </w>', 1)
('h a v e </w>', 1)
('N </w>', 1)
('a p p l e s </w>', 1)


In [17]:
def get_bpe_tokens(bpe_vocab):
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回，例如：
    输入 bpe_vocab=
    {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    得到
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('a', 2),
        ('m', 1),
        ('</w>', 5),
        ('y', 1),
        ('v', 1),
        ('e', 2),
        ('N', 1),
        ('l', 1),
        ('s', 1)
     ]

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """
    
    # TODO: 完成函数体（2分）
    bpe_tokens_as_dict = dict()
    
    for word, freq in bpe_vocab.items():
        word_split = word.split(' ')
        for key in word_split:
            bpe_tokens_as_dict[key] = bpe_tokens_as_dict.get(key,0)+freq
    bpe_tokens = list(bpe_tokens_as_dict.items())
    bpe_tokens.sort(reverse = True, key = lambda x: len(x[0]) if x[0][-4:]!='</w>' else len(x[0]) - 3 )

    return bpe_tokens

In [18]:
#此函数特别需要注意的一点是，由于我们要按照分词长度降序排序返回，可是词尾'</w>'在计算长度时结果为4，
#如果我们按照4处理，会影响排序，稍微影响到我们最后的结果（但是不影响正确性）
#不过只需要在排序时对lambda函数进行简单的微调即可处理这种特殊情况（函数返回前的最后一行即做了这个微调）

#以下是利用助教提供的例子进行的简单正确性测试，可以看到'</w>'是按照长度为1计算的，放在了正确的位置
bpe_vocab = {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
bpe_tokens = get_bpe_tokens(bpe_vocab)
for elem in bpe_tokens:
    print(elem)

('i</w>', 2)
('ha', 2)
('pp', 2)
('a', 2)
('m', 1)
('</w>', 5)
('y', 1)
('v', 1)
('e', 2)
('N', 1)
('l', 1)
('s', 1)


In [19]:
def print_bpe_tokenize(word, bpe_tokens):
    """
    根据按长度降序的BPE分词列表，将所给单词进行BPE分词，最后打印结果。
    
    思想是，对于一个待BPE分词的单词，按照长度顺序从列表中寻找BPE分词进行子串匹配，
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，
    或者剩余部分无法匹配（该部分整体由"<unknown>"代替）
    
    例1：
    输入 word="supermarket", bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]
    最终打印 "su per <unknown>"

    例2：
    输入 word="shanghai", bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]
    最终打印 "sh an g hai </w>"

    Args:
        word: str - 待分词的单词str
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的List
    """
    
    # TODO: 请尝试使用递归函数定义该分词过程（2分）
    def bpe_tokenize(sub_word):
        if sub_word == '':
            return sub_word
        for word, freq in bpe_tokens:
            for i in range(0, len(sub_word) - len(word) + 1):
                if sub_word[i: i+len(word)] == word:
                    return bpe_tokenize(sub_word[: i]) + word + ' ' + bpe_tokenize(sub_word[i+len(word):])
        return '<unknown> '

    res = bpe_tokenize(word+"</w>")
    print(res)

In [20]:
#按照助教提供提示的递归思路实现即可。以下是简单的正确性测试
word="supermarket"
bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]
print_bpe_tokenize(word,bpe_tokens)
word="shanghai"
bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]
print_bpe_tokenize(word,bpe_tokens)

su per <unknown> 
sh an g hai </w> 


开始读取数据集并训练BPE分词器

In [21]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")

Loaded training corpus.


In [22]:
training_iter_num = 300

training_bpe_vocab = build_bpe_vocab(training_corpus)
for i in range(training_iter_num):
    # TODO: 完成训练循环内的代码逻辑（2分）
    bigram_freq_as_dict = get_bigram_freq(training_bpe_vocab)
    bigram_freq_as_list = list(bigram_freq_as_dict.items())
    bigram_freq_as_list.sort(reverse = True, key = lambda x: x[1])
    bigram = bigram_freq_as_list[0][0]
    training_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(bigram, training_bpe_vocab)

training_bpe_tokens = get_bpe_tokens(training_bpe_vocab)

在每个循环内更新training_bpe_vocal.基本的思路是先从旧的training_bpe_vocab中提取bigram的频率，而得到的频率是以词典形式存储的。
我们需要将其转化成列表并按照频率进行降序排序，之后从列表的首位（第一个索引0）取出需要被合并的bigram（第二个索引0），
调用refresh_bpe_vocab_by_merging_bigram函数对training_bpe_vocab进行更新即可。

测试BPE分词器的分词效果

In [23]:
test_word = "naturallanguageprocessing"

print("naturallanguageprocessing 的分词结果为：")
print_bpe_tokenize(test_word, training_bpe_tokens)

naturallanguageprocessing 的分词结果为：
n atur al lan gu age pro ce s sing</w> 


In [24]:
#以下是我随意测试的一些其他分词结果
test_list = [
    "shanghaijiaotonguniversity",
    "no800dongchuanroad",
    "siyuanlakeistothesouthofbaoyuganglibrary",
    "enforcingmodularitywithclientsandcustomerservices",
    "afatcatsatonthemat",
    "blablablab@@@@@lablabla"
]
for elem in test_list:
    print_bpe_tokenize(elem,training_bpe_tokens)

s han g ha i j i a o ton g uni ver sity</w> 
no <unknown> don g c hu an ro ad</w> 
si y u an l ake i sto the sou tho f b a o y u g an g li b r ary</w> 
en for cin g mo du l ari t y with cli en t s an d cu s tom er ser vices</w> 
af at c at s ato n them at</w> 
b l ab l ab l ab <unknown> l ab l ab l a</w> 
